# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,f64,str,str,str,bool
"""../../../../da…","""33554733289795…",1.2927e19,"""68871599228682…","""17961173238645…","""99124901294654…",false
"""../../../../da…","""41612928036320…",1.4856e19,"""16237620892896…","""17961173238645…","""55318887053168…",true
"""../../../../da…","""16017655634561…",5.6458e17,"""13679798155767…","""17422534483443…","""99124901294654…",false
"""../../../../da…","""15087258745982…",4.0723e18,"""16237620892896…","""99934939659878…","""99124901294654…",false
"""../../../../da…","""12552201923913…",1.4677e19,"""68871599228682…","""99934939659878…","""96591623659206…",true
"""../../../../da…","""16877496767446…",1.6330e19,"""13679798155767…","""43336542392808…",null,false
"""../../../../da…","""15432601962466…",8.7675e18,"""13679798155767…","""99934939659878…","""96591623659206…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'filename': 'filename',
 'Customer_City': 'PREDICTOR_1',
 'Customer_CLV': 'PREDICTOR_2',
 'Customer_MaritalStatus': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""../../../../da…","""10462430955680…",1.2927e19,"""18408582867386…","""FirstMortgage3…","""76911208803436…","""Rejected"""
"""../../../../da…","""22674975296576…",1.4856e19,"""15663950308881…","""FirstMortgage3…","""84401571626636…","""Accepted"""
"""../../../../da…","""10171906171783…",5.6458e17,"""11532969266158…","""MoneyMarketSav…","""76911208803436…","""Rejected"""
"""../../../../da…","""16591873433243…",4.0723e18,"""15663950308881…","""BasicChecking""","""76911208803436…","""Rejected"""
"""../../../../da…","""15694743897750…",1.4677e19,"""18408582867386…","""BasicChecking""","""17668946058322…","""Accepted"""
"""../../../../da…","""16643256392340…",1.6330e19,"""11532969266158…","""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…","""14215739341929…",8.7675e18,"""11532969266158…","""BasicChecking""","""17668946058322…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""../../../../da…","""10459623694942…",1.2927e19,"""78918051986669…","""FirstMortgage3…","""12889841344540…","""Rejected"""
"""../../../../da…","""13799959378780…",1.4856e19,"""13729024578159…","""FirstMortgage3…","""97284147443536…","""Accepted"""
"""../../../../da…","""13034873962582…",5.6458e17,"""14007428750112…","""MoneyMarketSav…","""12889841344540…","""Rejected"""
"""../../../../da…","""10157614692407…",4.0723e18,"""13729024578159…","""BasicChecking""","""12889841344540…","""Rejected"""
"""../../../../da…","""84334439827201…",1.4677e19,"""78918051986669…","""BasicChecking""","""20534554823434…","""Accepted"""
"""../../../../da…","""13245149420483…",1.6330e19,"""14007428750112…","""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…","""32592130378790…",8.7675e18,"""14007428750112…","""BasicChecking""","""20534554823434…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

filename=filename
Customer_City=PREDICTOR_1
Customer_CLV=PREDICTOR_2
Customer_MaritalStatus=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""25324143153403…",1.2927e19,"""17010423473892…","""FirstMortgage3…","""11841913134983…","""Rejected"""
"""59140076077456…",1.4856e19,"""62536342548477…","""FirstMortgage3…","""15335398401203…","""Accepted"""
"""14919350980205…",5.6458e17,"""85529933933685…","""MoneyMarketSav…","""11841913134983…","""Rejected"""
"""17743825613434…",4.0723e18,"""62536342548477…","""BasicChecking""","""11841913134983…","""Rejected"""
"""13374402195380…",1.4677e19,"""17010423473892…","""BasicChecking""","""33681915992815…","""Accepted"""
"""15343704294132…",1.6330e19,"""85529933933685…","""UPlusFinPerson…",null,"""Rejected"""
"""10811742484641…",8.7675e18,"""85529933933685…","""BasicChecking""","""33681915992815…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'